'제74주년 경찰의 날 기념식 축사' 전문을 읽고 TF-IDF를 계산하세요.

전문은 http://www.korea.kr/archive/speechView.do?newsId=132031636 에서 읽을 수 있고,

해당 사이트에서 텍스트만 파일로 저장한 후 사용한다.



# 1) DataFrame을 생성

In [1]:
import pyspark
myConf=pyspark.SparkConf()
spark = pyspark.sql.SparkSession.builder\
    .master("local")\
    .appName("myApp")\
    .config(conf=myConf)\
    .getOrCreate()

In [2]:
import os
f_path=os.path.join('data','제74주년 경찰의 날 기념식 축사.txt')

In [3]:
from pyspark.sql import Row
myRdd = spark.sparkContext.textFile(f_path)
myRdd = myRdd.filter(lambda x : x != '').map(lambda x: Row(sent=x))
myDf = myRdd.toDF()
myDf.show()

+----------------------------------+
|                              sent|
+----------------------------------+
|  존경하는 국민 여러분, 경찰관 ...|
| 국민의 안전을 위해 밤낮없이 애...|
|오늘 홍조근정훈장을 받으신 중앙...|
|           사랑하는 경찰관 여러분,|
|여러분의 헌신적 노력으로 우리의...|
| 치안의 개선은 국민의 체감으로 ...|
| 한국을 찾는 외국 관광객들도 우...|
|   올해는 ‘경찰의 날’에 맞춰 국...|
|         자랑스러운 경찰관 여러분,|
| 경찰헌장은 “나라와 겨레를 위하...|
|    대한민국 경찰은 1945년 광복...|
| 임시정부 초대 경무국장 백범 김...|
|    광복 이후 6·25전쟁에서도 경...|
|그러나 잘못도 없지는 않았습니다...|
| 지금 경찰은 과거를 돌아보며 국...|
| 검경 수사권 조정과 자치경찰제 ...|
|정부는 경찰의 근무여건을 개선하...|
| 정부는 누구도 법 위에 군림하지...|
| 경찰헌장은 따뜻한 경찰, 의로운...|
|행사를 준비하신 민갑룡 경찰청장...|
+----------------------------------+
only showing top 20 rows



# 2) 단어로 분리해서, 출력

In [4]:
from pyspark.ml.feature import Tokenizer
tokenizer = Tokenizer(inputCol="sent", outputCol="words")
tokDf = tokenizer.transform(myDf)
for i in tokDf.select('words').collect():
    print(i.words)

['존경하는', '국민', '여러분,', '경찰관', '여러분,', '일흔네', '돌', '‘경찰의', '날’입니다.']
['국민의', '안전을', '위해', '밤낮없이', '애쓰시는', '전국의', '15만', '경찰관', '여러분께', '먼저', '감사를', '드립니다.', '전몰·순직', '경찰관들의', '고귀한', '희생에', '경의를', '표합니다.', '유가족', '여러분께', '위로의', '마음을', '전합니다.']
['오늘', '홍조근정훈장을', '받으신', '중앙경찰학교장', '이은정', '치안감님,', '근정포장을', '받으신', '광주남부경찰서', '김동현', '경감님을', '비롯한', '수상자', '여러분께', '각별한', '축하와', '감사를', '드립니다.', '또한', '경찰', '영웅으로', '추서되신', '차일혁,', '최중락님께', '국민의', '사랑을', '전해드립니다.']
['사랑하는', '경찰관', '여러분,']
['여러분의', '헌신적', '노력으로', '우리의', '치안은', '참', '좋아졌습니다.', '지난해', '범죄', '발생은', '2015년에', '비해', '15.1%', '줄었습니다.', '같은', '기간', '교통사고', '사망자는', '18.2%', '감소했습니다.']
['치안의', '개선은', '국민의', '체감으로', '나타나고', '있습니다.', '올해', '상반기', '국민의', '체감안전도는', '74.5점으로', '역대', '최고를', '기록했습니다.', '범죄안전도는', '처음으로', '80점을', '넘었습니다.']
['한국을', '찾는', '외국', '관광객들도', '우리의', '치안을', '가장', '좋게', '평가합니다.', '한국의', '무엇이', '좋았느냐는', '물음에', '외국', '관광객들은', '7년', '연속으로', '치안이', '가장', '좋았다고', '응답했습니다.', '개발도상국들은', '우리의', '경찰을', '모범으로', '삼으려', '합니다.']
['올해는', '

# 3) 불용어 구성, 출력 - 축사 전문에서 한글자로 된 단어를 찾아내 스스로 구성

In [5]:
stopwords = list()
for sent in tokDf.select('words').collect():
    words = sent.words
    for i in range(len(words)):
        if len(words[i]) == 1:
            if words[i] not in stopwords:
                stopwords.append(words[i])

In [6]:
print(stopwords)

['돌', '참', '더', '될', '그', '등', '법', '큰', '수']


# 4) 불용어 제거하고, 출력

In [7]:
from pyspark.ml.feature import StopWordsRemover
stop = StopWordsRemover(inputCol="words", outputCol="nostops")
stop.setStopWords(stopwords)
# for e in stop.getStopWords():
#     print (e, end="/")

StopWordsRemover_76bfbcf39389

In [8]:
stopDf=stop.transform(tokDf)
for i in stopDf.select('words', 'nostops').collect():
    print(i.words)
    print(">>", end=' ')
    print(i.nostops)
    print('\n')

['존경하는', '국민', '여러분,', '경찰관', '여러분,', '일흔네', '돌', '‘경찰의', '날’입니다.']
>> ['존경하는', '국민', '여러분,', '경찰관', '여러분,', '일흔네', '‘경찰의', '날’입니다.']


['국민의', '안전을', '위해', '밤낮없이', '애쓰시는', '전국의', '15만', '경찰관', '여러분께', '먼저', '감사를', '드립니다.', '전몰·순직', '경찰관들의', '고귀한', '희생에', '경의를', '표합니다.', '유가족', '여러분께', '위로의', '마음을', '전합니다.']
>> ['국민의', '안전을', '위해', '밤낮없이', '애쓰시는', '전국의', '15만', '경찰관', '여러분께', '먼저', '감사를', '드립니다.', '전몰·순직', '경찰관들의', '고귀한', '희생에', '경의를', '표합니다.', '유가족', '여러분께', '위로의', '마음을', '전합니다.']


['오늘', '홍조근정훈장을', '받으신', '중앙경찰학교장', '이은정', '치안감님,', '근정포장을', '받으신', '광주남부경찰서', '김동현', '경감님을', '비롯한', '수상자', '여러분께', '각별한', '축하와', '감사를', '드립니다.', '또한', '경찰', '영웅으로', '추서되신', '차일혁,', '최중락님께', '국민의', '사랑을', '전해드립니다.']
>> ['오늘', '홍조근정훈장을', '받으신', '중앙경찰학교장', '이은정', '치안감님,', '근정포장을', '받으신', '광주남부경찰서', '김동현', '경감님을', '비롯한', '수상자', '여러분께', '각별한', '축하와', '감사를', '드립니다.', '또한', '경찰', '영웅으로', '추서되신', '차일혁,', '최중락님께', '국민의', '사랑을', '전해드립니다.']


['사랑하는', '경찰관', '여러분,']
>> ['사랑하는', '경찰관', '여러분,']


['여러분의', '헌신적', '노력으로

# 5) TF-IDF를 계산하고, 출력

In [58]:
from pyspark.ml.feature import HashingTF, IDF

hashTF = HashingTF(inputCol="nostops", outputCol="hash")
hashDf = hashTF.transform(stopDf)

idf = IDF(inputCol="hash", outputCol="idf")
idfModel = idf.fit(hashDf)
idfDf = idfModel.transform(hashDf)
idfDf.select('idf').collect()

[Row(idf=SparseVector(262144, {162: 2.3979, 80732: 2.3979, 118171: 2.9632, 127225: 2.3979, 142775: 1.9924, 160086: 1.2993, 172380: 2.3979})),
 Row(idf=SparseVector(262144, {5341: 2.3979, 6304: 2.3979, 30732: 2.3979, 39431: 1.7047, 43098: 2.3979, 49855: 1.9924, 51468: 2.3979, 63600: 2.3979, 75300: 2.3979, 77757: 1.9924, 89318: 1.9924, 96799: 2.3979, 110980: 2.3979, 123553: 1.9924, 160081: 2.3979, 160086: 1.2993, 167255: 1.2993, 178931: 2.3979, 208192: 2.3979, 217323: 3.9849, 257249: 2.3979, 261393: 2.3979})),
 Row(idf=SparseVector(262144, {29823: 2.3979, 32228: 2.3979, 36822: 2.3979, 39431: 1.7047, 41144: 2.3979, 49855: 1.9924, 61014: 2.3979, 61103: 2.3979, 64713: 2.3979, 72971: 2.3979, 82902: 1.9924, 84159: 1.9924, 89227: 2.3979, 118725: 4.7958, 153204: 2.3979, 167255: 1.2993, 178229: 2.3979, 185989: 2.3979, 206065: 2.3979, 217323: 1.9924, 235673: 2.3979, 237148: 2.3979, 248593: 2.3979, 249689: 2.3979, 251574: 1.9924, 254458: 2.3979})),
 Row(idf=SparseVector(262144, {118171: 1.4816, 16

# 6) TF-IDF 컬럼을 features로 구성, 출력

In [60]:
idfDf = idfDf.withColumnRenamed('idf', 'features')
idfDf = idfDf.drop('hash')
idfDf.show(5, truncate=False)

+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|nostops                                                       

In [61]:
spark.stop()